In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
gameData = pd.read_csv('games.csv', dayfirst=False, parse_dates=[3])
playsData = pd.read_csv('plays.csv')
playersData = pd.read_csv('players.csv')
pffData = pd.read_csv('PFFScoutingData.csv')
trackingData2018 = pd.read_csv("tracking2018.csv")
trackingData2019 = pd.read_csv("tracking2019.csv")
trackingData2020 = pd.read_csv("tracking2020.csv")

In [3]:
def downcast(dataFrame):
    startMemoryOptimization = dataFrame.memory_usage().sum() / 1024 ** 2
    print('Memory Usage\033[1;31;43m Before \033[0;0mOptimization of Dataframe is:\033[1m {:.2f} MB \033[0m'.format(startMemoryOptimization))
    floatColumns = [c for c in dataFrame if dataFrame[c].dtype in ['float64']]
    intColumns = [c for c in dataFrame if dataFrame[c].dtype in ['int64']]
    objectColumns = [c for c in dataFrame if dataFrame[c].dtype in ['object']]
    dataFrame[floatColumns] = dataFrame[floatColumns].astype(np.float32)
    dataFrame[intColumns] = dataFrame[intColumns].astype(np.int32)
    for column in dataFrame[objectColumns].columns:
        if column =='date':
                dataFrame['date'] = pd.to_datetime(dataFrame['date'],format='%Y-%m-%d')
                
    endMemoryOptimization = dataFrame.memory_usage().sum() / 1024 ** 2
    print('Memory Usage\033[1;31;43m After \033[0;0mOptimization is:\033[1m {:.2f} MB \033[0m'.format(endMemoryOptimization))
    print('\033[1m'+'Compressed by:\033[1;30;45m {:.2f} % \033[0;0m'.format(100*(startMemoryOptimization - endMemoryOptimization) / startMemoryOptimization))
    return dataFrame


print('\n' + '\033[1;30;42m' + ' Game Data Compression: ' + '\033[0m')
gameData = downcast(gameData)
print('\n' + '\033[1;30;42m' + ' Plays Data Compression: ' + '\033[0m')
playsData = downcast(playsData)
print('\n' + '\033[1;30;42m' + ' Playres Data Compression: ' + '\033[0m')
playersData = downcast(playersData)
print('\n' + '\033[1;30;42m' + ' Tracking Data 2018 Compression: ' + '\033[0m')
trackingData2018 = downcast(trackingData2018)
print('\n' + '\033[1;30;42m' + ' Tracking Data 2019 Compression: ' + '\033[0m')
trackingData2019 = downcast(trackingData2019)
print('\n' + '\033[1;30;42m' + ' Tracking Data 2020 Compression: ' + '\033[0m')
trackingData2020 = downcast(trackingData2020)


 Game Data Compression: 
Memory Usage Before Optimization of Dataframe is: 0.04 MB 
Memory Usage After Optimization is: 0.03 MB 
Compressed by: 21.36 % 

 Plays Data Compression: 
Memory Usage Before Optimization of Dataframe is: 3.81 MB 
Memory Usage After Optimization is: 2.67 MB 
Compressed by: 30.00 % 

 Playres Data Compression: 
Memory Usage Before Optimization of Dataframe is: 0.15 MB 
Memory Usage After Optimization is: 0.13 MB 
Compressed by: 14.27 % 

 Tracking Data 2018 Compression: 
Memory Usage Before Optimization of Dataframe is: 1754.70 MB 
Memory Usage After Optimization is: 1169.80 MB 
Compressed by: 33.33 % 

 Tracking Data 2019 Compression: 
Memory Usage Before Optimization of Dataframe is: 1671.42 MB 
Memory Usage After Optimization is: 1114.28 MB 
Compressed by: 33.33 % 

 Tracking Data 2020 Compression: 
Memory Usage Before Optimization of Dataframe is: 1623.46 MB 
Memory Usage After Optimization is: 1082.31 MB 
Compressed by: 33.33 % 


In [4]:
trackingFull = pd.concat([trackingData2018, trackingData2019, trackingData2020], axis=0)
trackingFull.reset_index(drop=True, inplace=True)

In [6]:
mergedGameAndPlaysData = pd.merge(gameData, playsData, how='outer')

In [8]:
mergedGamePlaysAndTrackingFullData = pd.merge(mergedGameAndPlaysData,trackingFull, how ='outer')

In [10]:
mergedGamePlaysAndTrackingFullData["Result"] = mergedGamePlaysAndTrackingFullData["specialTeamsPlayType"]+ " - " + mergedGamePlaysAndTrackingFullData["specialTeamsResult"].map(str)
mergedGamePlaysAndTrackingFullData.drop(['specialTeamsPlayType','specialTeamsResult'],axis=1, inplace=True)

In [62]:
kickoffDataFrame = mergedGamePlaysAndTrackingFullData[['season', 'gameId','playId','possessionTeam','Result','kickerId',
                                                        'yardlineNumber','kickLength','event','nflId','displayName','position','kickReturnYardage']]

In [63]:
kickoffEvents = ['Kickoff - Touchback','Kickoff - Return','Kickoff - Muffed','Kickoff - Kickoff Team Recovery','Kickoff - Out of Bounds',
                 'Kickoff - Fair Catch','Kickoff - Downed']
kickoffEventsFrame = kickoffDataFrame.loc[kickoffDataFrame['Result'].isin(kickoffEvents)]

In [64]:
kickoffEventsFrame['event'].unique()

array(['None', 'kickoff', 'touchback', 'kick_received', 'first_contact',
       'tackle', 'kickoff_land', 'touchdown', 'onside_kick',
       'out_of_bounds', 'kickoff_play', 'free_kick', 'shift',
       'kick_recovered', 'fumble_defense_recovered', 'fumble', 'lateral',
       'penalty_flag', 'fumble_offense_recovered', 'fair_catch',
       'free_kick_play', 'punt_muffed', 'pass_outcome_caught',
       'punt_received', 'punt_land', 'handoff', 'drop_kick',
       'punt_downed', 'penalty_accepted', 'autoevent_kickoff', 'punt',
       'pass_shovel'], dtype=object)

In [65]:
kickoffEventsFrame.head().T

,0,1,2,3,4
season,2018,2018,2018,2018,2018
gameId,2018090600,2018090600,2018090600,2018090600,2018090600
playId,37,37,37,37,37
possessionTeam,PHI,PHI,PHI,PHI,PHI
Result,Kickoff - Touchback,Kickoff - Touchback,Kickoff - Touchback,Kickoff - Touchback,Kickoff - Touchback
kickerId,44966.0,44966.0,44966.0,44966.0,44966.0
yardlineNumber,35,35,35,35,35
kickLength,66.0,66.0,66.0,66.0,66.0
event,None,None,None,None,None
nflId,32361.0,32361.0,32361.0,32361.0,32361.0


In [201]:
onlyKickoffOf2018 = kickoffEventsFrame[(kickoffEventsFrame.season == 2018 ) & 
                                        (kickoffEventsFrame.event == 'kickoff') & 
                                        (kickoffEventsFrame.position == 'K') 
                                        ]

onlyKickoffTouchbackOf2018 = kickoffEventsFrame[(kickoffEventsFrame.season == 2018 ) & 
                                        (kickoffEventsFrame.event == 'touchback') & 
                                        (kickoffEventsFrame.position == 'K') 
                                        ]

onlyKickoffReturnOf2018 = kickoffEventsFrame[(kickoffEventsFrame.season == 2018 ) & 
                                        (kickoffEventsFrame.Result == 'Kickoff - Return') &
                                        (kickoffEventsFrame.event == 'kickoff') & 
                                        (kickoffEventsFrame.position == 'K') 
                                        ]

onlyKickoffOutOfBoundsOf2018 = kickoffEventsFrame[(kickoffEventsFrame.season == 2018 ) & 
                                        (kickoffEventsFrame.Result == 'Kickoff - Out of Bounds') &
                                        (kickoffEventsFrame.event == 'out_of_bounds') & 
                                        (kickoffEventsFrame.position == 'K') 
                                        ]

onlyKickoffOnsideKickOf2018 = kickoffEventsFrame[(kickoffEventsFrame.season == 2018 ) & 
                                        (kickoffEventsFrame.event == 'onside_kick') & 
                                        (kickoffEventsFrame.position == 'K') 
                                        ]

onlyKickoffTouchdownOf2018 = kickoffEventsFrame[(kickoffEventsFrame.season == 2018 ) & 
                                        (kickoffEventsFrame.event == 'touchdown') & 
                                        (kickoffEventsFrame.position == 'K') 
                                        ]


In [204]:
onlyKickoffOf2018Series = onlyKickoffOf2018.groupby(
    ['season', 'possessionTeam', 'displayName', 'position'], as_index=False).size()
onlyKickoffOf2018SeriesFrame = pd.DataFrame(onlyKickoffOf2018Series)

onlyKickoffOf2018SeriesFrame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': 'Kickoffs'}, inplace=True)

cleanKickoffOf2018FrameSeries = \
onlyKickoffOf2018SeriesFrame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    'Kickoffs'].sum().sort_values(ascending=False)
cleanKickoffOf2018FrameSeriesFrame = pd.DataFrame(cleanKickoffOf2018FrameSeries)
kickoff2018Ranking = cleanKickoffOf2018FrameSeriesFrame.reset_index()


###############################################################################################################################################################


onlyKickoffTouchbackOf2018Series = onlyKickoffTouchbackOf2018.groupby(
    ['season', 'possessionTeam', 'displayName', 'position'], as_index=False).size()
onlyKickoffTouchbackOf2018SeriesFrame = pd.DataFrame(onlyKickoffTouchbackOf2018Series)

onlyKickoffTouchbackOf2018SeriesFrame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': 'Kickoff TB'}, inplace=True)

cleanKickoffTouchbackOf2018FrameSeries = \
onlyKickoffTouchbackOf2018SeriesFrame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    'Kickoff TB'].sum().sort_values(ascending=False)
cleanKickoffTouchbackOf2018FrameSeriesFrame = pd.DataFrame(cleanKickoffTouchbackOf2018FrameSeries)
kickoffTouchback2018Ranking = cleanKickoffTouchbackOf2018FrameSeriesFrame.reset_index()



###############################################################################################################################################################


onlyKickoffReturnOf2018Series = onlyKickoffReturnOf2018.groupby(
    ['season', 'possessionTeam', 'displayName', 'position'], as_index=False).size()
onlyKickoffReturnOf2018SeriesFrame = pd.DataFrame(onlyKickoffReturnOf2018Series)

onlyKickoffReturnOf2018SeriesFrame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': 'Kickoff RT'}, inplace=True)

cleanKickoffReturnOf2018FrameSeries = \
onlyKickoffReturnOf2018SeriesFrame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    'Kickoff RT'].sum().sort_values(ascending=False)
cleanKickoffReturnOf2018FrameSeriesFrame = pd.DataFrame(cleanKickoffReturnOf2018FrameSeries)
kickoffReturn2018Ranking = cleanKickoffReturnOf2018FrameSeriesFrame.reset_index()



###############################################################################################################################################################


onlyKickoffOutOfBoundsOf2018Series = onlyKickoffOutOfBoundsOf2018.groupby(
    ['season', 'possessionTeam', 'displayName', 'position'], as_index=False).size()
onlyKickoffOutOfBoundsOf2018SeriesFrame = pd.DataFrame(onlyKickoffOutOfBoundsOf2018Series)

onlyKickoffOutOfBoundsOf2018SeriesFrame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': 'Kickoff OTB'}, inplace=True)

onlyKickoffOutOfBoundsOf2018FrameSeries = \
onlyKickoffOutOfBoundsOf2018SeriesFrame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    'Kickoff OTB'].sum().sort_values(ascending=False)
onlyKickoffOutOfBoundsOf2018FrameSeriesFrame = pd.DataFrame(onlyKickoffOutOfBoundsOf2018FrameSeries)
kickoffOutOfBounds2018Ranking = onlyKickoffOutOfBoundsOf2018FrameSeriesFrame.reset_index()


###############################################################################################################################################################


onlyKickoffOnsideKickOf2018Series = onlyKickoffOnsideKickOf2018.groupby(
    ['season', 'possessionTeam', 'displayName', 'position'], as_index=False).size()
onlyKickoffOnsideKickOf2018SeriesFrame = pd.DataFrame(onlyKickoffOnsideKickOf2018Series)

onlyKickoffOnsideKickOf2018SeriesFrame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': 'Kickoff OSK'}, inplace=True)

cleanKickoffOnsideKickOf2018FrameSeries = \
onlyKickoffOnsideKickOf2018SeriesFrame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    'Kickoff OSK'].sum().sort_values(ascending=False)
cleanKickoffOnsideKickOf2018FrameSeriesFrame = pd.DataFrame(cleanKickoffOnsideKickOf2018FrameSeries)
kickoffOnsideKick2018Ranking = cleanKickoffOnsideKickOf2018FrameSeriesFrame.reset_index()


###############################################################################################################################################################


onlyKickoffTouchdownOf2018Series = onlyKickoffTouchdownOf2018.groupby(
    ['season', 'possessionTeam', 'displayName', 'position'], as_index=False).size()
onlyKickoffTouchdownOf2018SeriesFrame = pd.DataFrame(onlyKickoffTouchdownOf2018Series)

onlyKickoffTouchdownOf2018SeriesFrame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': 'Kickoff TD'}, inplace=True)

cleanKickoffTouchdownOf2018Frame = \
onlyKickoffTouchdownOf2018SeriesFrame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    'Kickoff TD'].sum().sort_values(ascending=False)
cleanKickoffTouchdownOf2018FrameFrame = pd.DataFrame(cleanKickoffTouchdownOf2018Frame)
kickoffTouchdown2018Ranking = cleanKickoffTouchdownOf2018FrameFrame.reset_index()

In [205]:

mergeKickoffAndTouchbacks = pd.merge(kickoff2018Ranking,
                                     kickoffTouchback2018Ranking, 
                                     how='outer'
                                    )


In [206]:
mergeKickoffAndTouchbacks.replace(np.nan, 0,inplace=True)
mergeKickoffAndTouchbacks['Kickoff TB'] = mergeKickoffAndTouchbacks['Kickoff TB'].astype(int)

In [207]:
mergeKickoffAndTouchbacks['TB Accuracy %'] = (mergeKickoffAndTouchbacks['Kickoff TB'] / mergeKickoffAndTouchbacks['Kickoffs']).round(3) * 100

In [208]:
mergeKickoffTouchbackReturn = pd.merge(mergeKickoffAndTouchbacks,
                                     kickoffReturn2018Ranking, 
                                     how='outer'
                                    )

In [209]:

mergeKickoffTouchbackReturnOTB = pd.merge(mergeKickoffTouchbackReturn,
                                     kickoffOutOfBounds2018Ranking, 
                                     how='left'
                                    )
mergeKickoffTouchbackReturnOTB.replace(np.nan, 0,inplace=True)
mergeKickoffTouchbackReturnOTB['Kickoff OTB'] = mergeKickoffTouchbackReturnOTB['Kickoff OTB'].astype(int)

In [210]:
mergeKickoffTouchbackReturnOTBOSK = pd.merge(mergeKickoffTouchbackReturnOTB,
                                     kickoffOnsideKick2018Ranking, 
                                     how='left'
                                    )

mergeKickoffTouchbackReturnOTBOSK.replace(np.nan, 0,inplace=True)
mergeKickoffTouchbackReturnOTBOSK['Kickoff OSK'] = mergeKickoffTouchbackReturnOTBOSK['Kickoff OSK'].astype(int)

In [211]:


mergeKickoffTouchbackReturnOTBOSKTD = pd.merge(mergeKickoffTouchbackReturnOTBOSK,
                                     kickoffTouchdown2018Ranking, 
                                     how='left'
                                    )

mergeKickoffTouchbackReturnOTBOSKTD.replace(np.nan, 0,inplace=True)
mergeKickoffTouchbackReturnOTBOSKTD['Kickoff TD'] = mergeKickoffTouchbackReturnOTBOSKTD['Kickoff TD'].astype(int)

In [212]:
mergeKickoffTouchbackReturnOTBOSKTD

,Season,Player Name,Team,Position,Kickoffs,Kickoff TB,TB Accuracy %,Kickoff RT,Kickoff OTB,Kickoff OSK,Kickoff TD
0,2018,Wil Lutz,NO,K,101,64,63.4,37,0,0,0
1,2018,Harrison Butker,KC,K,101,65,64.4,32,3,1,0
2,2018,Ka'imi Fairbairn,HOU,K,91,58,63.7,30,0,1,0
3,2018,Justin Tucker,BAL,K,91,58,63.7,31,1,1,0
4,2018,Stephen Gostkowski,NE,K,90,48,53.3,40,0,0,0
5,2018,Cody Parkey,CHI,K,88,64,72.7,22,1,1,1
6,2018,Jason Myers,NYJ,K,82,60,73.2,22,0,0,0
7,2018,Jake Elliott,PHI,K,81,57,70.4,24,0,1,0
8,2018,Mason Crosby,GB,K,81,41,50.6,39,0,2,1
9,2018,Aldrick Rosas,NYG,K,81,51,63.0,26,3,4,0


In [214]:
onlyKickoffOf2019 = kickoffEventsFrame[(kickoffEventsFrame.season == 2019 ) & 
                                        (kickoffEventsFrame.event == 'kickoff') & 
                                        (kickoffEventsFrame.position == 'K') 
                                        ]

onlyKickoffTouchbackOf2019 = kickoffEventsFrame[(kickoffEventsFrame.season == 2019 ) & 
                                        (kickoffEventsFrame.event == 'touchback') & 
                                        (kickoffEventsFrame.position == 'K') 
                                        ]

onlyKickoffReturnOf2019 = kickoffEventsFrame[(kickoffEventsFrame.season == 2019) & 
                                        (kickoffEventsFrame.Result == 'Kickoff - Return') &
                                        (kickoffEventsFrame.event == 'kickoff') & 
                                        (kickoffEventsFrame.position == 'K') 
                                        ]

onlyKickoffOutOfBoundsOf2019 = kickoffEventsFrame[(kickoffEventsFrame.season == 2019) & 
                                        (kickoffEventsFrame.Result == 'Kickoff - Out of Bounds') &
                                        (kickoffEventsFrame.event == 'out_of_bounds') & 
                                        (kickoffEventsFrame.position == 'K') 
                                        ]

onlyKickoffOnsideKickOf2019 = kickoffEventsFrame[(kickoffEventsFrame.season == 2019) & 
                                        (kickoffEventsFrame.event == 'onside_kick') & 
                                        (kickoffEventsFrame.position == 'K') 
                                        ]

onlyKickoffTouchdownOf2019 = kickoffEventsFrame[(kickoffEventsFrame.season == 2019) & 
                                        (kickoffEventsFrame.event == 'touchdown') & 
                                        (kickoffEventsFrame.position == 'K') 
                                        ]


In [215]:
onlyKickoffOf2019Series = onlyKickoffOf2019.groupby(
    ['season', 'possessionTeam', 'displayName', 'position'], as_index=False).size()
onlyKickoffOf2019SeriesFrame = pd.DataFrame(onlyKickoffOf2019Series)

onlyKickoffOf2019SeriesFrame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': 'Kickoffs'}, inplace=True)

cleanKickoffOf2019FrameSeries = \
onlyKickoffOf2019SeriesFrame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    'Kickoffs'].sum().sort_values(ascending=False)
cleanKickoffOf2019FrameSeriesFrame = pd.DataFrame(cleanKickoffOf2019FrameSeries)
kickoff2019Ranking = cleanKickoffOf2019FrameSeriesFrame.reset_index()


###############################################################################################################################################################


onlyKickoffTouchbackOf2019Series = onlyKickoffTouchbackOf2019.groupby(
    ['season', 'possessionTeam', 'displayName', 'position'], as_index=False).size()
onlyKickoffTouchbackOf2019SeriesFrame = pd.DataFrame(onlyKickoffTouchbackOf2019Series)

onlyKickoffTouchbackOf2019SeriesFrame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': 'Kickoff TB'}, inplace=True)

cleanKickoffTouchbackOf2019FrameSeries = \
onlyKickoffTouchbackOf2019SeriesFrame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    'Kickoff TB'].sum().sort_values(ascending=False)
cleanKickoffTouchbackOf2019FrameSeriesFrame = pd.DataFrame(cleanKickoffTouchbackOf2019FrameSeries)
kickoffTouchback2019Ranking = cleanKickoffTouchbackOf2019FrameSeriesFrame.reset_index()



###############################################################################################################################################################


onlyKickoffReturnOf2019Series = onlyKickoffReturnOf2019.groupby(
    ['season', 'possessionTeam', 'displayName', 'position'], as_index=False).size()
onlyKickoffReturnOf2019SeriesFrame = pd.DataFrame(onlyKickoffReturnOf2019Series)

onlyKickoffReturnOf2019SeriesFrame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': 'Kickoff RT'}, inplace=True)

cleanKickoffReturnOf2019FrameSeries = \
onlyKickoffReturnOf2019SeriesFrame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    'Kickoff RT'].sum().sort_values(ascending=False)
cleanKickoffReturnOf2019FrameSeriesFrame = pd.DataFrame(cleanKickoffReturnOf2019FrameSeries)
kickoffReturn2019Ranking = cleanKickoffReturnOf2019FrameSeriesFrame.reset_index()



###############################################################################################################################################################


onlyKickoffOutOfBoundsOf2019Series = onlyKickoffOutOfBoundsOf2019.groupby(
    ['season', 'possessionTeam', 'displayName', 'position'], as_index=False).size()
onlyKickoffOutOfBoundsOf2019SeriesFrame = pd.DataFrame(onlyKickoffOutOfBoundsOf2019Series)

onlyKickoffOutOfBoundsOf2019SeriesFrame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': 'Kickoff OTB'}, inplace=True)

onlyKickoffOutOfBoundsOf2019FrameSeries = \
onlyKickoffOutOfBoundsOf2019SeriesFrame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    'Kickoff OTB'].sum().sort_values(ascending=False)
onlyKickoffOutOfBoundsOf2019FrameSeriesFrame = pd.DataFrame(onlyKickoffOutOfBoundsOf2019FrameSeries)
kickoffOutOfBounds2019Ranking = onlyKickoffOutOfBoundsOf2019FrameSeriesFrame.reset_index()


###############################################################################################################################################################


onlyKickoffOnsideKickOf2019Series = onlyKickoffOnsideKickOf2019.groupby(
    ['season', 'possessionTeam', 'displayName', 'position'], as_index=False).size()
onlyKickoffOnsideKickOf2019SeriesFrame = pd.DataFrame(onlyKickoffOnsideKickOf2019Series)

onlyKickoffOnsideKickOf2019SeriesFrame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': 'Kickoff OSK'}, inplace=True)

cleanKickoffOnsideKickOf2019FrameSeries = \
onlyKickoffOnsideKickOf2019SeriesFrame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    'Kickoff OSK'].sum().sort_values(ascending=False)
cleanKickoffOnsideKickOf2019FrameSeriesFrame = pd.DataFrame(cleanKickoffOnsideKickOf2019FrameSeries)
kickoffOnsideKick2019Ranking = cleanKickoffOnsideKickOf2019FrameSeriesFrame.reset_index()


###############################################################################################################################################################


onlyKickoffTouchdownOf2019Series = onlyKickoffTouchdownOf2019.groupby(
    ['season', 'possessionTeam', 'displayName', 'position'], as_index=False).size()
onlyKickoffTouchdownOf2019SeriesFrame = pd.DataFrame(onlyKickoffTouchdownOf2019Series)

onlyKickoffTouchdownOf2019SeriesFrame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': 'Kickoff TD'}, inplace=True)

cleanKickoffTouchdownOf2019Frame = \
onlyKickoffTouchdownOf2019SeriesFrame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    'Kickoff TD'].sum().sort_values(ascending=False)
cleanKickoffTouchdownOf2019FrameFrame = pd.DataFrame(cleanKickoffTouchdownOf2019Frame)
kickoffTouchdown2019Ranking = cleanKickoffTouchdownOf2019FrameFrame.reset_index()

In [216]:

mergeKickoffAndTouchbacks2019 = pd.merge(kickoff2019Ranking,
                                     kickoffTouchback2019Ranking, 
                                     how='outer'
                                    )


In [217]:
mergeKickoffAndTouchbacks2019.replace(np.nan, 0,inplace=True)
mergeKickoffAndTouchbacks2019['Kickoff TB'] = mergeKickoffAndTouchbacks2019['Kickoff TB'].astype(int)

In [218]:
mergeKickoffAndTouchbacks2019['TB Accuracy %'] = (mergeKickoffAndTouchbacks2019['Kickoff TB'] / mergeKickoffAndTouchbacks2019['Kickoffs']).round(3) * 100

In [220]:
mergeKickoffTouchbackReturn2019 = pd.merge(mergeKickoffAndTouchbacks2019,
                                     kickoffReturn2019Ranking, 
                                     how='outer'
                                    )

In [221]:

mergeKickoffTouchbackReturnOTB2019 = pd.merge(mergeKickoffTouchbackReturn2019,
                                     kickoffOutOfBounds2019Ranking, 
                                     how='left'
                                    )
mergeKickoffTouchbackReturnOTB2019.replace(np.nan, 0,inplace=True)
mergeKickoffTouchbackReturnOTB2019['Kickoff OTB'] = mergeKickoffTouchbackReturnOTB2019['Kickoff OTB'].astype(int)

In [222]:
mergeKickoffTouchbackReturnOTBOSK2019 = pd.merge(mergeKickoffTouchbackReturnOTB2019,
                                     kickoffOnsideKick2019Ranking, 
                                     how='left'
                                    )

mergeKickoffTouchbackReturnOTBOSK2019.replace(np.nan, 0,inplace=True)
mergeKickoffTouchbackReturnOTBOSK2019['Kickoff OSK'] = mergeKickoffTouchbackReturnOTBOSK2019['Kickoff OSK'].astype(int)

In [223]:


mergeKickoffTouchbackReturnOTBOSKTD2019 = pd.merge(mergeKickoffTouchbackReturnOTBOSK2019,
                                     kickoffTouchdown2019Ranking, 
                                     how='left'
                                    )

mergeKickoffTouchbackReturnOTBOSKTD2019.replace(np.nan, 0,inplace=True)
mergeKickoffTouchbackReturnOTBOSKTD2019['Kickoff TD'] = mergeKickoffTouchbackReturnOTBOSKTD2019['Kickoff TD'].astype(int)

In [224]:
mergeKickoffTouchbackReturnOTBOSKTD2019

,Season,Player Name,Team,Position,Kickoffs,Kickoff TB,TB Accuracy %,Kickoff RT,Kickoff OTB,Kickoff OSK,Kickoff TD
0,2019,Justin Tucker,BAL,K,105,55,52.4,48.0,0,0,1
1,2019,Harrison Butker,KC,K,94,57,60.6,34.0,0,1,0
2,2019,Wil Lutz,NO,K,92,71,77.2,19.0,0,0,1
3,2019,Dan Bailey,MIN,K,84,61,72.6,20.0,0,0,0
4,2019,Jason Myers,SEA,K,84,58,69.0,22.0,0,0,0
5,2019,Greg Zuerlein,LA,K,83,62,74.7,16.0,2,0,0
6,2019,Zane Gonzalez,ARI,K,82,45,54.9,35.0,2,0,0
7,2019,Ka'imi Fairbairn,HOU,K,80,41,51.2,36.0,0,1,0
8,2019,Mason Crosby,GB,K,79,49,62.0,28.0,1,0,0
9,2019,Jake Elliott,PHI,K,77,49,63.6,26.0,0,2,1


In [225]:
onlyKickoffOf2020 = kickoffEventsFrame[(kickoffEventsFrame.season == 2020) & 
                                        (kickoffEventsFrame.event == 'kickoff') & 
                                        (kickoffEventsFrame.position == 'K') 
                                        ]

onlyKickoffTouchbackOf2020 = kickoffEventsFrame[(kickoffEventsFrame.season == 2020) & 
                                        (kickoffEventsFrame.event == 'touchback') & 
                                        (kickoffEventsFrame.position == 'K') 
                                        ]

onlyKickoffReturnOf2020 = kickoffEventsFrame[(kickoffEventsFrame.season == 2020) & 
                                        (kickoffEventsFrame.Result == 'Kickoff - Return') &
                                        (kickoffEventsFrame.event == 'kickoff') & 
                                        (kickoffEventsFrame.position == 'K') 
                                        ]

onlyKickoffOutOfBoundsOf2020 = kickoffEventsFrame[(kickoffEventsFrame.season == 2020) & 
                                        (kickoffEventsFrame.Result == 'Kickoff - Out of Bounds') &
                                        (kickoffEventsFrame.event == 'out_of_bounds') & 
                                        (kickoffEventsFrame.position == 'K') 
                                        ]

onlyKickoffOnsideKickOf2020 = kickoffEventsFrame[(kickoffEventsFrame.season == 2020) & 
                                        (kickoffEventsFrame.event == 'onside_kick') & 
                                        (kickoffEventsFrame.position == 'K') 
                                        ]

onlyKickoffTouchdownOf2020 = kickoffEventsFrame[(kickoffEventsFrame.season == 2020) & 
                                        (kickoffEventsFrame.event == 'touchdown') & 
                                        (kickoffEventsFrame.position == 'K') 
                                        ]


In [227]:
onlyKickoffOf2020Series = onlyKickoffOf2020.groupby(
    ['season', 'possessionTeam', 'displayName', 'position'], as_index=False).size()
onlyKickoffOf2020SeriesFrame = pd.DataFrame(onlyKickoffOf2020Series)

onlyKickoffOf2020SeriesFrame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': 'Kickoffs'}, inplace=True)

cleanKickoffOf2020FrameSeries = \
onlyKickoffOf2020SeriesFrame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    'Kickoffs'].sum().sort_values(ascending=False)
cleanKickoffOf2020FrameSeriesFrame = pd.DataFrame(cleanKickoffOf2020FrameSeries)
kickoff2020Ranking = cleanKickoffOf2020FrameSeriesFrame.reset_index()


###############################################################################################################################################################


onlyKickoffTouchbackOf2020Series = onlyKickoffTouchbackOf2020.groupby(
    ['season', 'possessionTeam', 'displayName', 'position'], as_index=False).size()
onlyKickoffTouchbackOf2020SeriesFrame = pd.DataFrame(onlyKickoffTouchbackOf2020Series)

onlyKickoffTouchbackOf2020SeriesFrame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': 'Kickoff TB'}, inplace=True)

cleanKickoffTouchbackOf2020FrameSeries = \
onlyKickoffTouchbackOf2020SeriesFrame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    'Kickoff TB'].sum().sort_values(ascending=False)
cleanKickoffTouchbackOf2020FrameSeriesFrame = pd.DataFrame(cleanKickoffTouchbackOf2020FrameSeries)
kickoffTouchback2020Ranking = cleanKickoffTouchbackOf2020FrameSeriesFrame.reset_index()



###############################################################################################################################################################


onlyKickoffReturnOf2020Series = onlyKickoffReturnOf2020.groupby(
    ['season', 'possessionTeam', 'displayName', 'position'], as_index=False).size()
onlyKickoffReturnOf2020SeriesFrame = pd.DataFrame(onlyKickoffReturnOf2020Series)

onlyKickoffReturnOf2020SeriesFrame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': 'Kickoff RT'}, inplace=True)

cleanKickoffReturnOf2020FrameSeries = \
onlyKickoffReturnOf2020SeriesFrame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    'Kickoff RT'].sum().sort_values(ascending=False)
cleanKickoffReturnOf2020FrameSeriesFrame = pd.DataFrame(cleanKickoffReturnOf2020FrameSeries)
kickoffReturn2020Ranking = cleanKickoffReturnOf2020FrameSeriesFrame.reset_index()



###############################################################################################################################################################


onlyKickoffOutOfBoundsOf2020Series = onlyKickoffOutOfBoundsOf2020.groupby(
    ['season', 'possessionTeam', 'displayName', 'position'], as_index=False).size()
onlyKickoffOutOfBoundsOf2020SeriesFrame = pd.DataFrame(onlyKickoffOutOfBoundsOf2020Series)

onlyKickoffOutOfBoundsOf2020SeriesFrame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': 'Kickoff OTB'}, inplace=True)

onlyKickoffOutOfBoundsOf2020FrameSeries = \
onlyKickoffOutOfBoundsOf2020SeriesFrame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    'Kickoff OTB'].sum().sort_values(ascending=False)
onlyKickoffOutOfBoundsOf2020FrameSeriesFrame = pd.DataFrame(onlyKickoffOutOfBoundsOf2020FrameSeries)
kickoffOutOfBounds2020Ranking = onlyKickoffOutOfBoundsOf2020FrameSeriesFrame.reset_index()


###############################################################################################################################################################


onlyKickoffOnsideKickOf2020Series = onlyKickoffOnsideKickOf2020.groupby(
    ['season', 'possessionTeam', 'displayName', 'position'], as_index=False).size()
onlyKickoffOnsideKickOf2020SeriesFrame = pd.DataFrame(onlyKickoffOnsideKickOf2020Series)

onlyKickoffOnsideKickOf2020SeriesFrame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': 'Kickoff OSK'}, inplace=True)

cleanKickoffOnsideKickOf2020FrameSeries = \
onlyKickoffOnsideKickOf2020SeriesFrame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    'Kickoff OSK'].sum().sort_values(ascending=False)
cleanKickoffOnsideKickOf2020FrameSeriesFrame = pd.DataFrame(cleanKickoffOnsideKickOf2020FrameSeries)
kickoffOnsideKick2020Ranking = cleanKickoffOnsideKickOf2020FrameSeriesFrame.reset_index()


###############################################################################################################################################################


onlyKickoffTouchdownOf2020Series = onlyKickoffTouchdownOf2020.groupby(
    ['season', 'possessionTeam', 'displayName', 'position'], as_index=False).size()
onlyKickoffTouchdownOf2020SeriesFrame = pd.DataFrame(onlyKickoffTouchdownOf2020Series)

onlyKickoffTouchdownOf2020SeriesFrame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': 'Kickoff TD'}, inplace=True)

cleanKickoffTouchdownOf2020Frame = \
onlyKickoffTouchdownOf2020SeriesFrame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    'Kickoff TD'].sum().sort_values(ascending=False)
cleanKickoffTouchdownOf2020FrameFrame = pd.DataFrame(cleanKickoffTouchdownOf2020Frame)
kickoffTouchdown2020Ranking = cleanKickoffTouchdownOf2020FrameFrame.reset_index()

In [228]:

mergeKickoffAndTouchbacks2020 = pd.merge(kickoff2020Ranking,
                                     kickoffTouchback2020Ranking, 
                                     how='outer'
                                    )


In [229]:
mergeKickoffAndTouchbacks2020.replace(np.nan, 0,inplace=True)
mergeKickoffAndTouchbacks2020['Kickoff TB'] = mergeKickoffAndTouchbacks2020['Kickoff TB'].astype(int)

In [230]:
mergeKickoffAndTouchbacks2020['TB Accuracy %'] = (mergeKickoffAndTouchbacks2020['Kickoff TB'] / mergeKickoffAndTouchbacks2020['Kickoffs']).round(3) * 100

In [231]:
mergeKickoffTouchbackReturn2020 = pd.merge(mergeKickoffAndTouchbacks2020,
                                     kickoffReturn2020Ranking, 
                                     how='outer'
                                    )

In [232]:

mergeKickoffTouchbackReturnOTB2020 = pd.merge(mergeKickoffTouchbackReturn2020,
                                     kickoffOutOfBounds2020Ranking, 
                                     how='left'
                                    )
mergeKickoffTouchbackReturnOTB2020.replace(np.nan, 0,inplace=True)
mergeKickoffTouchbackReturnOTB2020['Kickoff OTB'] = mergeKickoffTouchbackReturnOTB2020['Kickoff OTB'].astype(int)

In [233]:
mergeKickoffTouchbackReturnOTBOSK2020 = pd.merge(mergeKickoffTouchbackReturnOTB2020,
                                     kickoffOnsideKick2020Ranking, 
                                     how='left'
                                    )

mergeKickoffTouchbackReturnOTBOSK2020.replace(np.nan, 0,inplace=True)
mergeKickoffTouchbackReturnOTBOSK2020['Kickoff OSK'] = mergeKickoffTouchbackReturnOTBOSK2020['Kickoff OSK'].astype(int)

In [235]:


mergeKickoffTouchbackReturnOTBOSKTD2020 = pd.merge(mergeKickoffTouchbackReturnOTBOSK2020,
                                     kickoffTouchdown2020Ranking, 
                                     how='left'
                                    )

mergeKickoffTouchbackReturnOTBOSKTD2020.replace(np.nan, 0,inplace=True)
mergeKickoffTouchbackReturnOTBOSKTD2020['Kickoff TD'] = mergeKickoffTouchbackReturnOTBOSKTD2020['Kickoff TD'].astype(int)

In [236]:
mergeKickoffTouchbackReturnOTBOSKTD2020

,Season,Player Name,Team,Position,Kickoffs,Kickoff TB,TB Accuracy %,Kickoff RT,Kickoff OTB,Kickoff OSK,Kickoff TD
0,2020,Tyler Bass,BUF,K,100,67,67.0,30.0,0,0,0
1,2020,Justin Tucker,BAL,K,95,65,68.4,30.0,0,0,0
2,2020,Daniel Carlson,LV,K,93,55,59.1,32.0,0,2,0
3,2020,Harrison Butker,KC,K,92,69,75.0,19.0,2,1,1
4,2020,Wil Lutz,NO,K,91,66,72.5,20.0,1,2,0
5,2020,Jason Myers,SEA,K,89,49,55.1,38.0,0,0,0
6,2020,Jason Sanders,MIA,K,84,64,76.2,18.0,1,6,0
7,2020,Cairo Santos,CHI,K,83,41,49.4,41.0,0,3,0
8,2020,Stephen Gostkowski,TEN,K,83,48,57.8,31.0,1,2,0
9,2020,Ka'imi Fairbairn,HOU,K,81,49,60.5,28.0,0,2,0


In [237]:
kickoffFull = pd.concat([mergeKickoffTouchbackReturnOTBOSKTD, mergeKickoffTouchbackReturnOTBOSKTD2019, mergeKickoffTouchbackReturnOTBOSKTD2020], axis=0)
kickoffFull.reset_index(drop=True, inplace=True)

In [238]:
kickoffFull

,Season,Player Name,Team,Position,Kickoffs,Kickoff TB,TB Accuracy %,Kickoff RT,Kickoff OTB,Kickoff OSK,Kickoff TD
0,2018,Wil Lutz,NO,K,101,64,63.4,37.0,0,0,0
1,2018,Harrison Butker,KC,K,101,65,64.4,32.0,3,1,0
2,2018,Ka'imi Fairbairn,HOU,K,91,58,63.7,30.0,0,1,0
3,2018,Justin Tucker,BAL,K,91,58,63.7,31.0,1,1,0
4,2018,Stephen Gostkowski,NE,K,90,48,53.3,40.0,0,0,0
5,2018,Cody Parkey,CHI,K,88,64,72.7,22.0,1,1,1
6,2018,Jason Myers,NYJ,K,82,60,73.2,22.0,0,0,0
7,2018,Jake Elliott,PHI,K,81,57,70.4,24.0,0,1,0
8,2018,Mason Crosby,GB,K,81,41,50.6,39.0,0,2,1
9,2018,Aldrick Rosas,NYG,K,81,51,63.0,26.0,3,4,0


In [239]:
kickoffFull.to_csv(r'/Users/kushtrivedi/Desktop/NFL/Kickoffs_2018_to_2020.csv', index=False)